```python
tf.keras.layers.Embedding(
    input_dim, output_dim, embeddings_initializer='uniform',
    embeddings_regularizer=None, activity_regularizer=None,
    embeddings_constraint=None, mask_zero=False, input_length=None, **kwargs
)
```
- Input_dim: số lượng vocab tối đa cần được ước lượng
- output_dim: kích thước embedding mong muốn
- input_length: số lượng maxlen của câu đầu vào.


## Cách hoạt động:

1 câu (max_len x 1) => One hot encoding (max_len x Vocab_size) => hiddenLayer E (Vocab_size x N) => output (max_len x N)

# 1. Import some lib

In [1]:
import io
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization

In [2]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

# dataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz", url,
#                                   untar=True, cache_dir='.',
#                                   cache_subdir='')

# dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
os.listdir('aclImdb')

['imdb.vocab', 'imdbEr.txt', 'README', 'test', 'train']

In [4]:
train_dir = os.path.join('aclImdb', 'train')
os.listdir(train_dir)

['labeledBow.feat',
 'neg',
 'pos',
 'unsup',
 'unsupBow.feat',
 'urls_neg.txt',
 'urls_pos.txt',
 'urls_unsup.txt']

In [5]:
# remove_dir = os.path.join(train_dir, 'unsup')
# shutil.rmtree(remove_dir)

In [6]:
batch_size = 1024
seed = 123
train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='training', seed=seed)
val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='validation', seed=seed)

Found 75000 files belonging to 3 classes.
Using 60000 files for training.
Found 75000 files belonging to 3 classes.
Using 15000 files for validation.


In [7]:
for text_batch, label_batch in train_ds.take(1):
  for i in range(5):
    print(label_batch[i].numpy(), text_batch.numpy()[i])

1 b"Ask yourself where she got the gun? Remember what she was taught about the mark's mindset when the con is over? The gun had blanks and it was provided to her from the very beginning.<br /><br />When the patient comes back at the end she was SUPPOSED to see him drive away in the red convertible and lead her to the gang splitting up her 80 thousand.<br /><br />The patient was in on the con from the beginning.<br /><br />Mantegna does not die in the end - the gun had blanks.<br /><br />There - enough spoilers for you there? This is why people are giving it such high ratings. It's extremely original because of the hidden ending and how it cons MOST of the audience."
2 b"For some reason, people seem to have a problem differentiating this movie from Trail of the Pink Panther.<br /><br />At any rate, this work does nothing but serve to remind us how sad the world is without Peter Sellers in it.<br /><br />They brought back the same old favorites from Trail (Dreyfus, Cato, Litton, etc.), b

In [8]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [10]:
# Embed a 1,000 word vocabulary into 5 dimensions.
embedding_layer = tf.keras.layers.Embedding(1000, 5)

In [11]:
result = embedding_layer(tf.constant([1, 2, 3]))
result.numpy()

array([[ 0.04159603,  0.01817093,  0.03314178, -0.00271792,  0.00815636],
       [-0.03416543,  0.0154368 ,  0.00639908,  0.02999461, -0.04190104],
       [ 0.00738442, -0.02693652,  0.00236541, -0.02436489,  0.04716805]],
      dtype=float32)

In [12]:
result = embedding_layer(tf.constant([[0, 1, 2], [3, 4, 5]]))
result.shape

TensorShape([2, 3, 5])

## 1.Text preprocessing

In [13]:
# Create a custom standardization function to strip HTML break tags '<br />

def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />',' ')
    return tf.strings.regex_replace(stripped_html, '[%s]' % re.escape(string.punctuation), '')

In [14]:
# Vocabulary size and number of words in a sequence.
vocab_size = 10000
sequence_length = 100

In [15]:
# Use the text vectorization layer to normalize, split, and map strings to
# integers. Note that the layer uses the custom standardization defined above.
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)
# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

## Create a classification model

In [16]:
embedding_dim=16

model = Sequential([
  vectorize_layer,
  Embedding(vocab_size, embedding_dim, name="embedding"),
  GlobalAveragePooling1D(),
  Dense(16, activation='relu'),
  Dense(1)
])

## Complile and train the model

In [17]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [18]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [19]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=[tensorboard_callback])

Epoch 1/15
59/59 [==============================] - 29s 462ms/step - loss: 0.5180 - accuracy: 0.1656 - val_loss: 0.2648 - val_accuracy: 0.1711
Epoch 2/15
59/59 [==============================] - 6s 104ms/step - loss: -0.1966 - accuracy: 0.1655 - val_loss: -0.7572 - val_accuracy: 0.1690
Epoch 3/15
59/59 [==============================] - 6s 108ms/step - loss: -1.5155 - accuracy: 0.1661 - val_loss: -2.3472 - val_accuracy: 0.1690
Epoch 4/15
59/59 [==============================] - 6s 103ms/step - loss: -3.6823 - accuracy: 0.1661 - val_loss: -5.1874 - val_accuracy: 0.1690
Epoch 5/15
59/59 [==============================] - 6s 101ms/step - loss: -7.6414 - accuracy: 0.1661 - val_loss: -10.2334 - val_accuracy: 0.1690
Epoch 6/15
59/59 [==============================] - 6s 98ms/step - loss: -14.2341 - accuracy: 0.1661 - val_loss: -18.1863 - val_accuracy: 0.1690
Epoch 7/15
59/59 [==============================] - 6s 99ms/step - loss: -24.1263 - accuracy: 0.1661 - val_loss: -29.6499 - val_accurac

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, 100)              0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 100, 16)           160000    
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160,289
Trainable params: 160,289
Non-trai

In [21]:
#docs_infra: no_execute
%load_ext tensorboard
%tensorboard --logdir logs

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 15996.

## Retrieve the trained word embeddings and save them to disk

In [22]:
weights = model.get_layer('embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [23]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

##### Using this 2 file to http://projector.tensorflow.org/


<img src="eda.png" />

# dimension = 16